In [56]:
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import numpy as np
import pandas as pd

#Change these file paths for each of the relevent autoregressive LLMs

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/protgpt2_unconditional/generation_metadata_protgpt2.csv"
len_dist_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/protgpt2_unconditional/uniref50_length_dist_protgpt2.json"

all_metadata_df = pd.read_csv(meta_data_filepath)
with open(len_dist_filepath, "r") as f:
  uniprot_length_dist =  json.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#need to make sure we take the exact length 100 sequences the we selected previously
fasta_filepath = "/content/drive/MyDrive/Generative_Models/utilities/fold_inputs/all_len100.fa"
len100s = []

with open(fasta_filepath, "r") as f:
  fasta_sequences = f.read()

for record in fasta_sequences.split('>')[1:]:
  sequence_id = record.split('\n')[0]
  if meta_data_filepath.split("_")[-1].split(".")[0] in sequence_id:
    print(sequence_id)
    len100s.append(sequence_id.split("_")[-1])
print(len100s)


In [58]:
!pip install biopython
#LLM outputs sometimes have artifacts. Correct these first.
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import re

def is_valid_protein(sequence):
  try:
    Seq(str(sequence))
    return True
  except ValueError:
    return False

all_metadata_df.loc[:,'generated_sequence'] = all_metadata_df.loc[:,'generated_sequence'].apply(lambda x: re.sub(r'[\s\d]+', '', str(x)))
all_metadata_df.loc[:,'generated_sequence'] = all_metadata_df.loc[all_metadata_df.loc[:,'generated_sequence'].apply(is_valid_protein),:]
all_metadata_df.loc[:,'length'] = all_metadata_df.loc[:,'generated_sequence'].apply(lambda x: len(x))
all_metadata_df.drop('conditions',inplace=True, axis=1)

In [63]:
subsampled_dfs = []
for length, num_rows in uniprot_length_dist:
  #need to make sure we take the exact length 100 sequences the we selected previously
  if length == 100:
    sampled_df = all_metadata_df.loc[all_metadata_df.entity_id.isin(len100s),:]
  else:
    sampled_df = all_metadata_df[all_metadata_df['length'] == length].sample(frac=1, random_state=42).reset_index(drop=True).head(num_rows)
  if len(sampled_df) < num_rows:
      print(f"Warning: Only {len(filtered_df)} rows available for length {length}. Sampling all available rows.")
  subsampled_dfs.append(sampled_df)

In [64]:
clean_df = pd.concat(subsampled_dfs, ignore_index=True)

In [66]:
clean_df.to_csv(meta_data_filepath.replace(".csv","_clean.csv"),index = False)